In [1]:
# Cemhan Kaan Özaltan
# 21902695

import numpy as np
from PIL import Image
import glob

def normalize_pcs(pcs):
    res = []
    for i in range(len(pcs)):
        res.append((pcs[i] - pcs[i].min()) / (pcs[i].max() - pcs[i].min()))
    return np.array(res) * 255

def reconstruct(img, eigvecs_set, means, k):
    res, copy = np.zeros((64,64,3)), np.copy(img)

    copy[:,:,0], copy[:,:,1], copy[:,:,2] = copy[:,:,0] - means['r'], copy[:,:,1] - means['g'], copy[:,:,2] - means['b']
    flat = []
    for i in range(3):
        flat.append(copy[:,:,i].flatten())
    flat_np = np.array(flat)
    for i in range(k):
        res[:,:,0] += (np.dot(flat_np[0], eigvecs_set['r'][i]) * eigvecs_set['r'][i]).reshape(64, 64)
        res[:,:,1] += (np.dot(flat_np[1], eigvecs_set['g'][i]) * eigvecs_set['g'][i]).reshape(64, 64)
        res[:,:,2] += (np.dot(flat_np[2], eigvecs_set['b'][i]) * eigvecs_set['b'][i]).reshape(64, 64)

    res[:,:,0], res[:,:,1], res[:,:,2] = res[:,:,0] + means['r'], res[:,:,1] + means['g'], res[:,:,2] + means['b']
    recons = Image.fromarray(res.astype(np.uint8), mode="RGB")
    recons.save(f'./recons_k{k}.jpg') # change this path if needed

In [2]:
# question 1.1
img_names = [i for i in glob.glob("./afhq_dog/*.jpg")] # change this path if needed
img_count = len(img_names)
channels = {'r': [], 'g': [], 'b': []}

for img_name in img_names:
    img = np.array(Image.open(img_name).resize((64,64), Image.Resampling.BILINEAR))
    channels['r'].append(img[:,:,0].flatten())
    channels['g'].append(img[:,:,1].flatten())
    channels['b'].append(img[:,:,2].flatten())

r, g, b = np.array(channels['r']), np.array(channels['g']), np.array(channels['b'])
mean_r, mean_g, mean_b = np.mean(r, axis=0), np.mean(g, axis=0), np.mean(b, axis=0)
r, g, b = r - mean_r, g - mean_g, b - mean_b

r_cov, g_cov, b_cov = np.dot(r.T, r) / img_count, np.dot(g.T, g) / img_count, np.dot(b.T, b) / img_count
eigvals_r, eigvecs_r = np.linalg.eig(r_cov)
eigvals_g, eigvecs_g = np.linalg.eig(g_cov)
eigvals_b, eigvecs_b = np.linalg.eig(b_cov)
eigvecs_r, eigvecs_b, eigvecs_g = eigvecs_r.T, eigvecs_b.T, eigvecs_g.T

pve_sums = {'r': 0, 'g': 0, 'b': 0}

print("10 best PVEs")
for i in range(10):
    pve_r, pve_g, pve_b = eigvals_r[i] / eigvals_r.sum(), eigvals_g[i] / eigvals_g.sum(), eigvals_b[i] / eigvals_b.sum()
    pve_sums['r'] += pve_r
    pve_sums['g'] += pve_g
    pve_sums['b'] += pve_b
    print(f"Red channel's PVE #{i + 1}: {pve_r}")
    print(f"Green channel's PVE #{i + 1}: {pve_g}")
    print(f"Blue channel's PVE #{i + 1}: {pve_b}\n")

print(f"PVE sums\nRed: {pve_sums['r']}\nGreen: {pve_sums['g']}\nBlue: {pve_sums['b']}\n")

count_r, count_g, count_b = 10, 10, 10 # 10 PCEs already calculated

while pve_sums['r'] < 0.7:
    pve_sums['r'] += eigvals_r[count_r] / eigvals_r.sum()
    count_r += 1

while pve_sums['g'] < 0.7:
    pve_sums['g'] += eigvals_g[count_g] / eigvals_g.sum()
    count_g += 1

while pve_sums['b'] < 0.7:
    pve_sums['b'] += eigvals_b[count_b] / eigvals_b.sum()
    count_b += 1

print("Number of PVEs needed for 0.7 for the red channel:", count_r)
print("Number of PVEs needed for 0.7 for the green channel:", count_g)
print("Number of PVEs needed for 0.7 for the blue channel:", count_b)

10 best PVEs
Red channel's PVE #1: 0.2150928295705464
Green channel's PVE #1: 0.2004593939345657
Blue channel's PVE #1: 0.2299724795720598

Red channel's PVE #2: 0.13543590648775305
Green channel's PVE #2: 0.13767809111140247
Blue channel's PVE #2: 0.13678519667322256

Red channel's PVE #3: 0.07504975573273252
Green channel's PVE #3: 0.07695367428026625
Blue channel's PVE #3: 0.07034022694974615

Red channel's PVE #4: 0.051732173422999836
Green channel's PVE #4: 0.05397123455088882
Blue channel's PVE #4: 0.05356417895492482

Red channel's PVE #5: 0.04228943097350487
Green channel's PVE #5: 0.042918749347138276
Blue channel's PVE #5: 0.039821729668577326

Red channel's PVE #6: 0.024583277160715355
Green channel's PVE #6: 0.026022591594315067
Blue channel's PVE #6: 0.02373237907459678

Red channel's PVE #7: 0.021772692745900574
Green channel's PVE #7: 0.02142652916670169
Blue channel's PVE #7: 0.02099218512082034

Red channel's PVE #8: 0.019898578692023182
Green channel's PVE #8: 0.02081

In [3]:
# question 1.2
pcs = {
    'r': normalize_pcs(eigvecs_r[0:10,:].reshape(10, 64, 64)),
    'g': normalize_pcs(eigvecs_g[0:10,:].reshape(10, 64, 64)),
    'b': normalize_pcs(eigvecs_b[0:10,:].reshape(10, 64, 64))
}

for i in range(10):
    img = np.zeros((64,64,3))
    for j, channel in enumerate(pcs):
        img[:,:,j] = pcs[channel][i]

    final_img = Image.fromarray(img.astype(np.uint8), mode='RGB')
    final_img.save(f'./img_pc{i + 1}.jpg') # change this path if needed

In [4]:
# question 1.3
mean_r, mean_g, mean_b = mean_r.reshape(64, 64), mean_g.reshape(64, 64), mean_b.reshape(64, 64)
sample_img =np.array(Image.open('./afhq_dog/flickr_dog_000002.jpg').resize((64,64), Image.Resampling.BILINEAR)) # change this path if needed

eigvecs_set = {'r': eigvecs_r, 'g': eigvecs_g, 'b': eigvecs_b}
means = {'r': mean_r, 'g': mean_g, 'b': mean_b}
for k in [1, 50, 250, 500, 1000, 4096]:
    reconstruct(sample_img, eigvecs_set, means, k)